In [1]:
import numpy as np
import pandas as pd
import statistics
import matplotlib.pyplot as plt
from pandas.plotting import autocorrelation_plot
from statsmodels.graphics.tsaplots import plot_pacf
from matplotlib import pyplot
from statsmodels.tsa.arima_model import ARIMA 
from sklearn.metrics import mean_squared_error

In [2]:
#read file
df_data = pd.read_csv("data/dataset_mood_smartphone.csv")
df_data.head()

,Unnamed: 0,id,time,variable,value
0,1,AS14.01,2014-02-26 13:00:00.000,mood,6.0
1,2,AS14.01,2014-02-26 15:00:00.000,mood,6.0
2,3,AS14.01,2014-02-26 18:00:00.000,mood,6.0
3,4,AS14.01,2014-02-26 21:00:00.000,mood,7.0
4,5,AS14.01,2014-02-27 09:00:00.000,mood,6.0


In [3]:
prev_day="2014-02-26"
new_data=pd.DataFrame(columns=['id', 'time', 'var',"value"])
temp_list=[]
for index, row in df_data.iterrows():
    date=row["time"].split(" ")[0]
    if date != prev_day:
        new_data.loc[index]=[df_data.loc[index-1]["id"], prev_day , df_data.loc[index-1]["variable"], statistics.mean(temp_list)]
        temp_list=[row["value"]]
    else:
        temp_list.append(row["value"])
    prev_day=date

In [ ]:
ids = list(np.unique(np.array(df_data["id"])))

In [ ]:
error_ids = []
for id in ids:
    df_mood_id = new_data.loc[(new_data['var'] == "mood") & (new_data['id']==id)]
    df_time_value = df_mood_id[['id','time',"value"]].reset_index()
    df_time_value['time'] = pd.to_datetime(df_time_value['time'],format="%Y-%m-%d")

    X = []

    prev_index = 0
    for i, row in df_time_value.iterrows():
        delta = (df_time_value['time'].iloc[prev_index] - df_time_value['time'].iloc[i]).days
        if delta < -1:
            for n in range(abs(delta)-1):
                X.append(None)
            X.append(df_time_value['value'].iloc[i])
        elif delta > 0:
            for n in range(365-delta-1):
                X.append(None)
            X.append(df_time_value['value'].iloc[i])
        else:
            X.append(df_time_value['value'].iloc[i])
        prev_index = i

    #Set up the input for ARIMA
    # index = df_time_value[["time"]].values
    # data_np = df_time_value["value"].values
    series = pd.Series(X)
    X= series.values
    print(X)
#     np.savetxt("dataset_arima.txt", X)
    X = [x for x in X if str(x) != 'nan']
    
    size = int(len(X) * 0.66)
    train_val, test = X[0:size], X[size:len(X)]
    
    size_train = int(len(train_val) * 0.75)
    train, val = train_val[0:size_train], train_val[size_train:len(train_val)]
    

#   hyperparameter selection
    history = [x for x in train_val]
    print(len(history))
    range_p = [0,1,2,3,4]
    range_d = [0,1]
    range_q = [0,1,2,3,4]
    hyperparameter_error = []
    for p in range_p:
        for d in range_d:
            for q in range_q:
                try:
                    predictions = []
                    history = [x for x in train_val]
                    for t in range(len(val)):
                        model = ARIMA(history, order=(p,d,q))
                        model_fit = model.fit(disp=-1, transparams=False)
                        output = model_fit.forecast()
                        yhat = output[0]
                        predictions.append(yhat)
                        obs = val[t]
                        history.append(obs)
                    error = mean_squared_error(val, predictions)
                    hyperparameter_error.append([error,p,d,q])
                    print(p,d,q, error)
                except:
                    pass
    best_hyperparameter = [1000,0,0,0]
    
    print(hyperparameter_error)
    for i in hyperparameter_error:
        if i[0] < best_hyperparameter[0]:
            best_hyperparameter = i
    
    print('Error: \t'+ str(best_hyperparameter[0]))
    print('Best hyperparameter (p,d,q):\t',best_hyperparameter[1:4])

    
    history = [x for x in train_val]
    print(len(history))
    
    predictions = list()
    try:
        for c in range(len(test)):
            #order=(p,d,q) 
            model = ARIMA(history, order=(best_hyperparameter[1], best_hyperparameter[2], best_hyperparameter[3]))
            model_fit = model.fit(disp=-1,transparams=False)
            output = model_fit.forecast()
            yhat = output[0]
            predictions.append(yhat)
            obs = test[c]
            history.append(obs)
            #print('predicted=%f, expected=%f' % (yhat, obs))
        error = mean_squared_error(test, predictions)
    except:
        pass
#     plt.plot(test, predictions)
#     plt.show()
    print('Test MSE: %.3f' % error)
    error_ids.append([id,error, best_hyperparameter[0], best_hyperparameter[1], best_hyperparameter[2], best_hyperparameter[3]])
print(error_ids)

[6.25       6.33333333        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan 6.2
 6.4        6.8        6.         6.75       6.6        7.
 6.4        8.         7.5        7.4        6.         6.5
 6.4        6.2        6.8        6.5        6.5        6.6
 7.4        7.4        7.         7.25       8.         7.2
 6.4        7.2        6.6        6.8        7.8        7.25
 7.6        7.4        7.6        7.6        7.2        7.6
 7.5        7.6        7.         7.8        8.         7.6
 8.         8.        ]
31
0 0 0 0.27547912248550144
0 0 1 0.2527313551998657
0 0 2 0.2700267278027337
0 0 3 0.28822681819256335
0 0 4 0.3683439713902402
0 1 0 0.3844462674500691
0 1 1 0.37823141913098524
0 1 2 0.2544251559207925
1 0 0 0.2562222104477806


C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\tsa\kalmanf\kalmanfilter.py:649: RuntimeWarning: overflow encountered in square
  R_mat, T_mat)
C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\base\model.py:508: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\base\model.py:508: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\base\model.py:508: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\base\model.py:508: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
C:\ProgramData\Anacon

1 0 1 0.27090280797886523


C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\base\model.py:488: HessianInversionWarning: Inverting hessian failed, no bse or cov_params available
  'available', HessianInversionWarning)
C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\base\model.py:508: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\base\model.py:508: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\base\model.py:488: HessianInversionWarning: Inverting hessian failed, no bse or cov_params available
  'available', HessianInversionWarning)
C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\base\model.py:508: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", Converg

1 0 2 18.416890855938583
1 1 0 0.36631886501914734


...

C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\base\model.py:488: HessianInversionWarning: Inverting hessian failed, no bse or cov_params available
  'available', HessianInversionWarning)
C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\base\model.py:488: HessianInversionWarning: Inverting hessian failed, no bse or cov_params available
  'available', HessianInversionWarning)
C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\base\model.py:488: HessianInversionWarning: Inverting hessian failed, no bse or cov_params available
  'available', HessianInversionWarning)
C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\base\model.py:488: HessianInversionWarning: Inverting hessian failed, no bse or cov_params available
  'available', HessianInversionWarning)
C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\base\model.py:488: HessianInversionWarning: Inverting hessian failed, no bse or cov_params available
  'available', HessianInversionWarning)
C:\ProgramData\Anaco

4 0 3 1.2325946769653275


C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\base\model.py:488: HessianInversionWarning: Inverting hessian failed, no bse or cov_params available
  'available', HessianInversionWarning)
C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\base\model.py:488: HessianInversionWarning: Inverting hessian failed, no bse or cov_params available
  'available', HessianInversionWarning)
C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\base\model.py:508: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\base\model.py:488: HessianInversionWarning: Inverting hessian failed, no bse or cov_params available
  'available', HessianInversionWarning)
C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\base\model.py:508: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning

4 1 0 0.7913674447669248


C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\base\model.py:488: HessianInversionWarning: Inverting hessian failed, no bse or cov_params available
  'available', HessianInversionWarning)
C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\base\model.py:508: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)


In [ ]:
error

In [ ]:
df_mood_id = new_data.loc[(new_data['var'] == "mood")]
df_time_value = df_mood_id[['id','time',"value"]].reset_index()
df_time_value['time'] = pd.to_datetime(df_time_value['time'],format="%Y-%m-%d")

X = []

id = 'AS14.01'
prev_index = 0
for i, row in df_time_value.iterrows():
    delta = (df_time_value['time'].iloc[prev_index] - df_time_value['time'].iloc[i]).days
    if delta < -1:
        for n in range(abs(delta)-1):
            X.append(None)
        X.append(df_time_value['value'].iloc[i])
    elif delta > 0:
        for n in range(365-delta-1):
            X.append(None)
        X.append(df_time_value['value'].iloc[i])
    else:
        X.append(df_time_value['value'].iloc[i])
    prev_index = i

#Set up the input for ARIMA
# index = df_time_value[["time"]].values
# data_np = df_time_value["value"].values
series = pd.Series(X)
X= series.values
X = [x for x in X if str(x) != 'nan']
# X = [x for x in X]
size = int(len(X) * 0.66)
train, test = X[0:size], X[size:len(X)]

history = [x for x in train]
predictions = list()
for t in range(len(test)):
    #order=(p,d,q) 
    model = ARIMA(history, order=(1,1,1))#tunne parameters!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
    model_fit = model.fit(disp=0)
    output = model_fit.forecast()
    yhat = output[0]
    predictions.append(yhat)
    obs = test[t]
    history.append(obs)
    print('predicted=%f, expected=%f' % (yhat, obs))
error = mean_squared_error(test, predictions)
print('Test MSE: %.3f' % error)

In [ ]:
df_time_value.shape